# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
data.race.value_counts()

b    2435
w    2435
Name: race, dtype: int64

In [4]:
# number of callbacks for white-sounding names
data[data.race=='w'].call.sum()/len(data[data.race=='w'])

0.09650924024640657

In [5]:
# number of callbacks for white-sounding names
data[data.race=='b'].call.sum()/len(data[data.race=='b'])

0.06447638603696099

In [6]:
data.columns

Index(['id', 'ad', 'education', 'ofjobs', 'yearsexp', 'honors', 'volunteer',
       'military', 'empholes', 'occupspecific', 'occupbroad', 'workinschool',
       'email', 'computerskills', 'specialskills', 'firstname', 'sex', 'race',
       'h', 'l', 'call', 'city', 'kind', 'adid', 'fracblack', 'fracwhite',
       'lmedhhinc', 'fracdropout', 'fraccolp', 'linc', 'col', 'expminreq',
       'schoolreq', 'eoe', 'parent_sales', 'parent_emp', 'branch_sales',
       'branch_emp', 'fed', 'fracblack_empzip', 'fracwhite_empzip',
       'lmedhhinc_empzip', 'fracdropout_empzip', 'fraccolp_empzip',
       'linc_empzip', 'manager', 'supervisor', 'secretary', 'offsupport',
       'salesrep', 'retailsales', 'req', 'expreq', 'comreq', 'educreq',
       'compreq', 'orgreq', 'manuf', 'transcom', 'bankreal', 'trade',
       'busservice', 'othservice', 'missind', 'ownership'],
      dtype='object')

<div class="span5 alert alert-success">

1. What test is appropriate for this problem? Does CLT apply?
<br></br>
We can do a two sample proportion test.  The CLT would apply because there are large numbers of samples for both black and white candidates.
<br></br>
<br></br>
2. What are the null and alternate hypotheses?
<br></br>
The null hypothesis is that there is no difference between the callback rate for black sounding names compared with white sounding names.  The alternate hypothesis is that the callback rate for black sounding names is lower than that for white sounding names.
</div>

In [7]:
# Frequentist approach
from statsmodels.stats.proportion import proportions_ztest
count_black = data[data.race=='b'].call.sum()
count_white = data[data.race=='w'].call.sum()

nobs_black = len(data[data.race=='b'])
nobs_white = len(data[data.race=='w'])

prop_black = count_black/nobs_black
prop_white = count_white/nobs_white

sd = np.sqrt(((prop_black * (1-prop_black))/nobs_black) + ((prop_white * (1-prop_white))/nobs_white))
moe = 1.96*sd

ci = [(prop_black-prop_white)-moe, (prop_black-prop_white)+moe]

count = np.array([count_black, count_white])
nobs = np.array([nobs_black, nobs_white])
stat, pval = proportions_ztest(count, nobs, alternative = 'smaller')

print('Frequentist Approach')
print('point estimate:', (prop_black-prop_white))
print('margin of error:', moe)
print('confidence interval:', ci)
print('pvalue:', pval)

#Bootstrap approach
#create sample of 1000 draws from total population:
diffs = []
for i in range(1, 10001):
    black_sample = data.sample(n = 2435, replace = True)
    white_sample = data.sample(n = 2435, replace = True)
    
    black_prop = black_sample.call.sum()/len(black_sample)
    white_prop = white_sample.call.sum()/len(white_sample)
    
    diff_prop = black_prop - white_prop
    diffs.append(diff_prop)

print('')
print('Bootstrap Approach')
p_value = (diffs <= (prop_black-prop_white)).sum()/10000
print('p-value', p_value)

Frequentist Approach
point estimate: -0.032032854209445585
margin of error: 0.015255406349886438
confidence interval: [-0.047288260559332024, -0.016777447859559147]
pvalue: 1.9919434187925383e-05

Bootstrap Approach
p-value 0.0


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
4. Write a story describing the statistical significance in the context or the original problem.
<br></br>
Both approaches show that if there truly is no difference in proportion of callbacks, it would be very unlikely to see results as extreme as the one we observed by chance.  Per the frequentist approach, the probability is 0.0002 and per the boostrap approachwith 10000 experiments, the probability is 0.  Thus, the alternate hypothesis is more likely than the null hypothesis.  It is very likely that black sounding names truly do get fewer call backs.
<br></br>
<br></br>
5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?
<br></br>
It doesn't necessarily mean that race/name is the most important factor.  However, given the p-values, race/name is a strong factor.  Its unlikely other factors are as important as race/name because the experiment randomized resume assignment to a name.  This random selection makes it unlikely for black candidates to look very different from white candidates.  We can do additional analysis to ensure that the two groups have no meaningful differences.
</div>